In [ ]:
from datetime import timedelta

import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# [0. ¿Qué es eso del churn?](#0.-¿Qué-es-eso-del-churn?)

El "churn" es un término que se utiliza en el ámbito de los negocios para referirse a la pérdida de clientes o usuarios. En el caso de una empresa, el churn se refiere a la pérdida de clientes que dejan de comprar productos o servicios de la empresa. Por ejemplo, En el caso de una empresa de telecomunicaciones, el churn se refiere a la pérdida de clientes que dejan de utilizar los servicios de la empresa.

En castellano habitualmente usamos "baja" para referirnos a este fenómeno.

Para una empresa, el churn es un problema importante, ya que la pérdida de clientes puede tener un impacto negativo en los ingresos y en la rentabilidad de la empresa. Por lo tanto, es importante que las empresas identifiquen a los clientes que tienen más probabilidades de abandonar la empresa y tomen medidas para intentar retenerlos.

# [1. Dataset](#1)

Vamos a utilizar este conjunto de datos de ventas de una tienda online del Reino Unido, entre el 01/12/2009 y el 09/12/2011. La empresa principalmente vende artículos de regalo. Muchos de los clientes de la empresa son mayoristas.

Podéis descargar el dataset desde [aquí](https://www.kaggle.com/datasets/mashlyn/online-retail-ii-uci?resource=download). Si os habéis clonado el repositiorio, en la carpeta `data` ya tenéis el archivo `online_retail_II.csv.zip`. Descomprimid el archivo y guardad el archivo `online_retail_II.csv` en la carpeta `data`.

In [ ]:
# Cargar el dataset
df = pd.read_csv('../data/online_retail_II.csv')

# Limpieza de datos
df = df.dropna(subset=['Customer ID'])
df = df[df['Quantity'] > 0]
df['TotalPrice'] = df['Quantity'] * df['Price']

# Convertimos la columna 'InvoiceDate' a datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Extraemos el año y el mes de la columna 'InvoiceDate'
df['YearMonth'] = df['InvoiceDate'].dt.to_period('M')

# Casteamos la columna 'Customer ID' a int y la renombramos a 'CustomerID'
df['CustomerID'] = df['Customer ID'].astype(int)
df = df.drop(columns='Customer ID')

# Generamos una lista de meses únicos
unique_months = df['YearMonth'].unique()

In [ ]:
df.head()

¿Cómo haríais para intentar predecir el churn de los clientes de esta empresa?

# [2. RFM](#2)

El análisis RFM es una técnica de análisis para identificar a los clientes que tienen más probabilidades de abandonar la empresa. RFM son las siglas de Recency, Frequency y Monetary Value.

El principio de funcionamiento análisis RFM es que a través de estos tres indicadores es posible modelizar el comportamiento de los clientes de una forma standar para cualquier tipo de negocio.   

Vamos a entender cómo se calculan estos tres indicadores.

## [2.1. Recency](#2.1)

La "recencia" es una medida de cuánto tiempo ha pasado desde la última interacción de un cliente con la empresa. En este caso, la última compra.

In [ ]:
df.query("CustomerID == 13837")

¿Cual sería la recencia del cliente 13837 el 1 de Junio de 2011 (2011-06-01)?

In [ ]:
ultima_compra = df.query("CustomerID == 13837 and InvoiceDate < '2011-06-01'")['InvoiceDate'].max()
recencia = pd.Timestamp('2011-06-01') - ultima_compra
recencia

## [2.2. Frequency](#2.2)

La "frecuencia" es una medida de cuántas veces un cliente ha interactuado con la empresa en un período de tiempo determinado.

¿Cuál sería la frecuencia del cliente 13837 el 1 de Junio de 2011 (2011-06-01)?

In [ ]:
frecuencia_absoluta = df.query("CustomerID == 13837 and InvoiceDate < '2011-06-01'")['Invoice'].nunique()
frecuencia_absoluta

En el análisis RFM la frecuencia se calcula de forma absoluta como nº de ocurrecias. Sin embargo, para muchos la frecuencia es una medida de ocurrencias por unidad de tiempo.
Realmente nada nos impide calcular la frecuencia de forma relativa, es decir, el número de ocurrencias por unidad de tiempo.

In [ ]:
ocurrencias_ultimos_6_meses = df.query("CustomerID == 13837 and InvoiceDate >= '2010-12-01' and InvoiceDate < '2011-06-01'")['Invoice'].nunique()
frecuencia_mensual_ultimos_6_meses = ocurrencias_ultimos_6_meses / 6
frecuencia_mensual_ultimos_6_meses

## [2.3. Monetary value](#2.3)

El "valor monetario" es una medida de cuánto ha gastado un cliente en un período de tiempo determinado. Habitualmente se calcula como la suma de los importes de las compras.

¿Cuál sería el valor monetario del cliente 13837 el 1 de Junio de 2011 (2011-06-01)?

In [ ]:
valor_monetario = df.query("CustomerID == 13837 and InvoiceDate < '2011-06-01'")['TotalPrice'].sum()
print(f"{valor_monetario:.2f} €")

La propiedad de monetary value como la suma de las compras es un consenso, pero en realidad esta magnitud pretende representar es el valor que tiene un cliente para la empresa. Por ejemplo, un cliente que haya comprado al por mayor y haya pagado el precio de venta al por menor, repercutirá un mayor beneficio a la empresa que un cliente que haya comprado al por menor.

## [2.4. Temporalidad](#2.4)

El análisis RFM puede ser tanto estático como dinámico. Mientras que el análisis estático se realiza en un momento concreto teniendo en cuenta toda la historia de interacciones, el análisis dinámico se realiza en un periodo de tiempo determinado.

Según el objetivo buscado se puede realizar un análisis RFM estático o dinámico.   


Dado que nuestro objetivo actual es predecir el churn de los clientes, vamos a realizar un análisis RFM dinámico mensual. De esta forma podremos analizar la evolución de los clientes en el tiempo y el número de datos será mayor.   


Dependiendo del tipo de negocio, el análisis RFM puede ser más o menos sensible a la temporalidad. Por ejemplo, en un negocio de venta de ropa de temporada, la temporalidad es un factor muy importante. En cambio, en un negocio de venta de productos de limpieza, la temporalidad es menos importante. En cualquier caso, es un hiperparámetro que hay que elegir cuidadosamente.

## [2.5. Cálculo](#2.5)

In [ ]:
# Funcion para calcular RFM para un mes dado
def calculate_rfm(data, snapshot_date):
    rfm = data.groupby('CustomerID').agg({
        'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
        'Invoice': 'nunique',
        'TotalPrice': 'sum'
    }).reset_index()
    rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']
    return rfm

# Lista para almacenar los datos RFM mensuales
rfm_monthly = []

# Iterar sobre cada mes para calcular RFM
for month in unique_months:
    month_data = df[df['YearMonth'] <= month]
    snapshot_date = (month + 1).to_timestamp()
    rfm = calculate_rfm(month_data, snapshot_date)
    rfm['YearMonth'] = month
    rfm_monthly.append(rfm)

# concatena los datos de RFM mensuales
rfm_monthly_df = pd.concat(rfm_monthly)

In [ ]:
rfm_monthly_df.query("CustomerID == 13837")

# [3. Churn](#4)

## [3.1. Definición](#3.1)

Como ya hemos comentado antes, el churn es la pérdida de clientes. Sin embargo, esto no siempre fácil de definir según el tipo de empresa:

- En empresas con modelos de suscripción o contrato, como una empresa telefónica, habitualmente definir el churn es tan sencillo como determinar cuando el cliente ha realizado la gestión para dar de baja el servicio.
- En empresas de venta de productos, como la que estamos analizando, el churn es más complicado de definir. ¿Cuándo consideramos que un cliente ha abandonado la empresa? ¿Cuándo ha pasado un tiempo sin comprar? ¿Cuándo ha pasado un tiempo sin interactuar con la empresa?

Para simplificar el análisis, vamos a considerar que un cliente ha abandonado la empresa si no ha realizado ninguna compra en los últimos 6 meses.

In [ ]:
# Un cliente se considera churn si no ha realizado una compra en los últimos 6 meses, es decir, 180 días (aprox.)
churn_threshold = 180

## [3.2. Cálculo](#3.2)

Puesto que vamos a realizar un análisis RFM dinámico mensual, vamos a calcular el churn mensual. Es decir, vamos a calcular cuántos clientes han abandonado la empresa en un mes determinado.

Al definir el churn como la falta de compras en los últimos 6 meses, puede darse la situación de que un cliente haya abandonado la empresa y vuelva a comprar. En este caso el cliente se considerará como churn durante algunos meses y después se considerará de nuevo como cliente activo.

In [ ]:
# Función para calcular el churn
def calculate_churn(customer_id, current_month, df):
    next_purchase = df[(df['CustomerID'] == customer_id) &
                       (df['InvoiceDate'] > current_month.to_timestamp())]
    if next_purchase.empty:
        return 1
    days_to_next_purchase = (next_purchase['InvoiceDate'].min() - current_month.to_timestamp()).days
    return int(days_to_next_purchase > churn_threshold)

# Calcular el churn para cada fila en los datos mensuales de RFM
rfm_monthly_df['Churn'] = rfm_monthly_df.apply(
    lambda row: calculate_churn(row['CustomerID'], row['YearMonth'], df), axis=1)

# El cálculo de churn tarda un poco, por lo que guardamos los datos en un archivo CSV para no esperar
rfm_monthly_df.to_csv('../data/rfm_monthly.csv', index=False)

In [ ]:
# Cargamos los datos de RFM mensuales con el calculo de churn
rfm_monthly_df = pd.read_csv('../data/rfm_monthly.csv')
rfm_monthly_df['YearMonth'] = pd.to_datetime(rfm_monthly_df['YearMonth']).dt.to_period('M')

In [ ]:
rfm_monthly_df.query("CustomerID == 13837")

In [ ]:
rfm_monthly_df.value_counts('Churn')

In [ ]:
# Entre los últimos 180 días (chrun_treshold), la etiqueta churn no es precisa ya que no tenemos los datos futuros para completar los 180 días de datos por delante
cutoff_date = df['InvoiceDate'].max() - timedelta(days=churn_threshold)
cutoff_period = cutoff_date.to_period('M')
# Filtrar los datos para mantener solo los datos de entrenamiento etiquetados con churn fiable
rfm_monthly_df = rfm_monthly_df[rfm_monthly_df["YearMonth"]>=cutoff_period]

In [ ]:
rfm_monthly_df.value_counts('Churn')

## [3.3 Predicción](#3.3)

In [ ]:
# Para hacer una predicción de churn, necesitamos predecir el target (churn) con los datos del mes anterior
rfm_monthly_df_lagged = rfm_monthly_df.copy()
rfm_monthly_df_lagged["Churn_lagged"] = rfm_monthly_df_lagged.groupby("CustomerID")["Churn"].shift(-1)
rfm_monthly_df_lagged = rfm_monthly_df_lagged.dropna()

In [ ]:
# Preparación los datos para la predicción
X = rfm_monthly_df_lagged.drop(columns=['CustomerID', 'YearMonth', 'Churn', 'Churn_lagged'])
y = rfm_monthly_df_lagged['Churn_lagged']

# Normalización las features numéricas
scaler = StandardScaler()
X = scaler.fit_transform(X)

# División los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenamiento un modelo de Gradient Boosting
model = GradientBoostingClassifier()
model.fit(X_train, y_train)

# Evaluación del modelo
y_pred = model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
# print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
# Print matriz de confusión con labels
cm = confusion_matrix(y_test, y_pred)
print(pd.DataFrame(cm, columns=['Precicción no churn', 'Precicción churn'], index=['Real no churn', 'Real churn']))

Con esto tendríamos una predicción de los clientes que abandonarán la empresa el mes siguiente. Aunque la precisión parece alta, en realidad el resultado no es tan bueno como parece. ¿Por qué?
- Estamos prediciendo datos de clientes que ya se han dado de baja, lo cual es un patrón facilmente identificable por un modelo y no se debería predecir. Esto está mejorando artificialmente la precisión del modelo.
- Los falsos positivos son muy altos, más que los verdaderos positivos. Puesto que el objetivo es actuar sobre los clientes que van a abandonar la empresa, un alto número de falsos positivos puede ser muy costoso para la empresa.

Por otro lado, el análisis RFM ser puede utilzar de otras formas a parte de la predicción directa del churn. Veamos algunas de ellas.

# [4. RFM Scoring](#4)

Los valores RFM son unidades de medida estándar, cálculadas de forma homogénea para todos los clientes. Esto nos permite comparar a los clientes entre sí y establecer un ranking de los clientes más valiosos para la empresa.

Una forma sencilla de crear un ranking es obtener percentiles de los valores RFM. En este caso, vamos a suponer que queremos dividir a los clientes en 4 grupos:
- Clientes oro. Los clientes que tienen un valor RFM alto son los más valiosos para la empresa.
- Clientes plata. Los clientes que tienen un valor RFM medio.
- Clientes bronce. Los clientes que tienen un valor RFM bajo.
- Clientes plomo. Los clientes que tienen un valor RFM muy bajo.

In [ ]:
# Función para asignar puntuaciones RFM basadas en cuartiles
def assign_rfm_scores(df):
    df['R_Quartile'] = pd.qcut(df['Recency'], 4, labels=False, duplicates="drop") + 1
    df['F_Quartile'] = pd.qcut(df['Frequency'], 4, labels=False, duplicates="drop") + 1
    df['M_Quartile'] = pd.qcut(df['Monetary'], 4, labels=False, duplicates="drop") + 1

    # Invertir la puntuación de Recency
    df['R_Quartile'] = 5 - df['R_Quartile']

    return df

# Asignar puntuaciones RFM mes a mes
rfm_monthly_df = rfm_monthly_df.groupby('YearMonth', as_index=False).apply(assign_rfm_scores).reset_index(drop=True)

# Crear una puntuación RFM combinada de forma que la máxima puntuación (mejores clientes) sea 4*3 = 12
rfm_monthly_df['RFM_Score'] = 4*3 - (rfm_monthly_df['R_Quartile'] + rfm_monthly_df['F_Quartile'] + rfm_monthly_df['M_Quartile'])

In [ ]:
rfm_monthly_df.query("CustomerID == 13837")

In [ ]:
rfm_monthly_df.value_counts('RFM_Score').sort_index()

De la misma forma que hemos decidido dividir a los clientes en 4 grupos igualmente grandes (cuartiles), podríamos dividirlos en grupos heterogéneos. Por ejemplo, podríamos dividir a los clientes en 3 grupos:
- Clientes VIP. Los clientes que tienen un valor Recencia menor a 10 días, una Frecuencia mayor a 10 compras y un Valor Monetario mayor a 1000€.
- Clientes Regulares. Los clientes que tienen un valor Recencia menor a 30 días, una Frecuencia mayor a 5 compras y un Valor Monetario mayor a 500€.
- Clientes en riesgo. El resto de clientes.

Este tipo de clasificaciones resultan especialmente útiles si tenemos conocimientos específicos del negocio. Por ejemplo, si sabemos que los clientes que compran más de 10 veces al mes son los más valiosos para la empresa, podemos establecer un umbral de 10 compras al mes para clasificar a los clientes.

# [5. RFM Clustering](#5)

Realmente el scoring que hemos realizado en el apartado anterior es un tipo de clustering manual. En tal caso, ¿por qué no utilizar un algoritmo de clustering para realizar la clasificación de los clientes?

In [ ]:
# seleccionando las columnas RFM para el clustering
rfm_features = rfm_monthly_df[['Recency', 'Frequency', 'Monetary']]

# Elbow Analysis (Criterio del codo)
sse = []
k_range = range(1, 11)
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init="auto")
    kmeans.fit(rfm_features)
    sse.append(kmeans.inertia_)

# Graficar la curva del codo
plt.figure(figsize=(10, 6))
plt.plot(k_range, sse, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Sum of Squared Errors (SSE)')
plt.title('Elbow Method For Optimal k')
plt.show()

# Del gráfico del codo, elegir el número óptimo de clusters
optimal_k = 4

# Aplicando el clustering KMeans con el número de clusters elegido
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init="auto")
rfm_monthly_df['Cluster'] = kmeans.fit_predict(rfm_features)

In [ ]:
rfm_monthly_df.head()

In [ ]:
# ¿Cómo se relacionan los clusters creados con la puntuación RFM?
rfm_monthly_df.groupby('Cluster').agg({"RFM_Score": "mean"})

Si pudieramos hacer un gráfico para ver los clustereos de los clientes en función de los valores RFM, ¿cómo sería?

In [ ]:
# Reduce las features RFM a 2 dimensiones usando PCA
pca = PCA(n_components=2)
rfm_monthly_df[['PCA1', 'PCA2']] = pca.fit_transform(rfm_monthly_df[['Recency', 'Frequency', 'Monetary']])

# Parche para graficar los datos con Plotly
rfm_monthly_df["YearMonth"] = rfm_monthly_df["YearMonth"].astype(str)

# Crea un gráfico de dispersión de Plotly
fig = px.scatter(rfm_monthly_df,
                 x='PCA1',
                 y='PCA2',
                 color='RFM_Score',
                 hover_data=['CustomerID', 'YearMonth', 'RFM_Score'],
                 title='Customer Segments based on RFM Scores',
                 labels={'PCA1': 'Principal Component 1', 'PCA2': 'Principal Component 2'})

fig.show()

In [ ]:
rfm_monthly_df.head()

# [6. Conclusiones](#6)

- El "churn", en casteallano "baja", se define como la pérdida de clientes.
- El análisis RFM es una técnica de análisis para identificar a los clientes que tienen más probabilidades de abandonar la empresa.
- La ventaja principal del análisis RFM es que es una técnica sencilla, standard, fácil de entender y fácil de implementar.
- El análisis RFM puede ser tanto estático como dinámico.
- El análisis RFM puede ser utilizado para predecir el churn de los clientes, pero también para clasificar a los clientes en función de su valor para la empresa.